In [72]:
num_sample = 3000
num_burn = 500
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 1

In [73]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1,2,0.1,2,0.1,-2,0.1,2,0.1,0.1])

Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 9


100%|██████████| 3500/3500 [00:19<00:00, 176.05it/s]


(0.6020869497680362,
 0.41613874999999995,
 0.53946875,
 0.34431253882120677,
 0.36620294601895875)

In [74]:
# from drnet import expit

# L_chain = np.random.binomial(n=1, p=0.5, size=(1000, sample_size, 3))
# linpred = (
#         gamma[0]
#         + gamma[1] * L_chain[:, :, 0]
#         + gamma[3] * L_chain[:, :, 1]
#         + gamma[5] * L_chain[:, :, 2]
#     )
# prob_a = expit(linpred)
# A_chain = np.zeros(L_chain[:,:,0].shape)
# for i in range(L_chain.shape[0]):
#     A_chain[i] = np.random.binomial(n=1, p=prob_a[i], size=sample_size)

# linpred = (
#     beta[0]
#     + beta[1] * A_chain 
#     + beta[3] * L_chain[:, :, 0]
#     + beta[5] * L_chain[:, :, 1] 
#     + beta[7] * L_chain[:, :, 2]
# )
# probs = expit(linpred)
# Y_chain = np.zeros(L_chain[:,:,0].shape)
# for i in range(L_chain.shape[0]):
#     Y_chain[i] = np.random.binomial(n=1, p=probs[i], size=sample_size)

# L_chain.shape, A_chain.shape, Y_chain.shape

In [75]:
from drnet import expit
from sklearn.linear_model import LogisticRegression

def get_prob(L, Y, A):
    x_a = [L[:, 0], np.zeros(L.shape[0]), 
           L[:, 1], np.zeros(L.shape[0]), 
           L[:, 2], np.zeros(L.shape[0]), 
           np.zeros(L.shape[0])]
    
    # train lostic regression
    model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000)
    model.fit(np.array(x_a).T, A)
    
    gamma = np.zeros(8)
    gamma[0] = model.intercept_[0]
    gamma[1:] = model.coef_[0]

    #gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1]) 

    # train outcome model
    x_y = [A, np.zeros(L.shape[0]), 
           L[:, 0], np.zeros(L.shape[0]), 
           L[:, 1], np.zeros(L.shape[0]), 
           L[:, 2], np.zeros(L.shape[0]),
           np.zeros(L.shape[0])]
    model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000)
    model.fit(np.array(x_y).T, Y)

    beta = np.zeros(10)
    beta[0] = model.intercept_[0]
    beta[1:] = model.coef_[0]

    outcome1 = expit(
        beta[0]
        + beta[1] * 1 + beta[2] * 0
        + beta[3] * L[:, 0] + beta[4] * 0
        + beta[5] * L[:, 1] + beta[6] * 0
        + beta[7] * L[:, 2] + beta[8] * 0
        + beta[9] * 0
    )

    outcome0 = expit(
        beta[0]
        + beta[1] * 0 + beta[2] * 0
        + beta[3] * L[:, 0] + beta[4] * 0
        + beta[5] * L[:, 1] + beta[6] * 0
        + beta[7] * L[:, 2] + beta[8] * 0
        + beta[9] * 0
    )
    
    # x_y_pred = np.array(x_y).T
    # x_y_pred[:,0] = 1
    # outcome1 = model.predict_proba(x_y_pred)[:, 1]
    # x_y_pred[:,0] = 0
    # outcome0 = model.predict_proba(x_y_pred)[:, 1]

    linpred = (
            gamma[0]
            + gamma[1] * L[:, 0] + gamma[2] * 0
            + gamma[3] * L[:, 1] + gamma[4] * 0
            + gamma[5] * L[:, 2] + gamma[6] * 0
            + gamma[7] * 0
        )
    
    probs = expit(linpred)

    w = (A==1) / probs
    #w_norm = w/np.sum(w)*len(w) if np.sum(w) > 0 else 0
    #w_norm = A # probs
    gamma_1 = ((Y-outcome1) * w + outcome1).mean()
    gamma_1_var = ((Y-outcome1) * w + outcome1).var()

    w = (1-A) / (1-probs)
    #w_norm = w/np.sum(w)*len(w) if np.sum(w) > 0 else 0
    #w_norm = (1-A) # (1-probs)
    gamma_0 = ((Y-outcome0) * w + outcome0).mean()
    gamma_0_var = ((Y-outcome0) * w + outcome0).var()

    return gamma_1, gamma_0, gamma_1_var, gamma_0_var

gamma_1_est, gamma_0_est = [], []
gamma_1_var, gamma_0_var = [], []
for i in range(Y_chain.shape[0]):
    gamma_1, gamma_0, gamma_1_var_, gamma0_var_ = get_prob(L_chain[i], Y_chain[i], A_chain[i])
    gamma_1_est.append(gamma_1)
    gamma_0_est.append(gamma_0)
    gamma_1_var.append(gamma_1_var_)
    gamma_0_var.append(gamma0_var_)



print(np.mean(gamma_1_est), np.mean(gamma_0_est))
print(np.var(gamma_1_est)*sample_size, np.var(gamma_0_est)*sample_size, np.mean(gamma_1_var), np.mean(gamma_0_var))
#get_prob(L_chain[2], Y_chain[2], A_chain[2])
print(np.var(gamma_1_est), np.var(gamma_0_est))

0.7559590208037564 0.42931397691389006
1.1708868893130784 0.5267030794703267 1.1488458905462207 0.5068988013680441
0.001463608611641348 0.0006583788493379084
